In [1]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_lake")

df = db_handler.fetch_data(table_name="kr_stock_price_detail")

column_index = df.columns

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 57 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        200 non-null    int64              
 1   ticker                    200 non-null    object             
 2   rprs_mrkt_kor_name        200 non-null    object             
 3   new_hgpr_lwpr_cls_code    200 non-null    object             
 4   mxpr_llam_cls_code        200 non-null    object             
 5   crdt_able_yn              200 non-null    object             
 6   stck_mxpr                 200 non-null    object             
 7   elw_pblc_yn               200 non-null    object             
 8   prdy_clpr_vrss_oprc_rate  200 non-null    object             
 9   crdt_rate                 200 non-null    object             
 10  marg_rate                 200 non-null    object             
 11  lwpr_vrss_prpr     

,id,ticker,rprs_mrkt_kor_name,new_hgpr_lwpr_cls_code,mxpr_llam_cls_code,crdt_able_yn,stck_mxpr,elw_pblc_yn,prdy_clpr_vrss_oprc_rate,crdt_rate,marg_rate,lwpr_vrss_prpr,lwpr_vrss_prpr_sign,prdy_clpr_vrss_lwpr_rate,stck_lwpr,hgpr_vrss_prpr,hgpr_vrss_prpr_sign,prdy_clpr_vrss_hgpr_rate,stck_hgpr,oprc_vrss_prpr,oprc_vrss_prpr_sign,mang_issu_yn,divi_app_cls_code,short_over_yn,mrkt_warn_cls_code,invt_caful_yn,stange_runup_yn,ssts_hot_yn,low_current_yn,vi_cls_code,short_over_cls_code,stck_llam,new_lstn_cls_name,vlnt_deal_cls_name,flng_cls_name,revl_issu_reas_name,mrkt_warn_cls_name,stck_sdpr,bstp_cls_code,stck_prdy_clpr,insn_pbnt_yn,fcam_mod_cls_name,stck_prpr,prdy_vrss,prdy_vrss_sign,prdy_ctrt,acml_tr_pbmn,acml_vol,prdy_vrss_vol_rate,bstp_kor_isnm,sltr_yn,trht_yn,oprc_rang_cont_yn,vlnt_fin_cls_code,stck_oprc,prdy_vol,updated_at
0,1,005930,KOSPI200,신고가,NaN,Y,104600,Y,0.75,20.00,20.00,100,2,-1.12,79600,-1500,5,0.87,81200,-1400,5,N,,N,00,N,N,,N,N,,56400,,,NaN,NaN,NaN,80500,005930,80500,N,NaN,79700,-800,5,-0.99,1672547819584,20898386,81.57,전기·전자,N,N,N,N,81100,25619036,2025-09-21 18:41:43.356719+00:00
1,2,000660,KOSPI200,신고가,NaN,Y,458500,Y,-0.42,20.00,20.00,5000,2,-1.42,348000,-7000,5,1.98,360000,1500,2,N,,N,00,N,N,,N,N,,247500,,,NaN,NaN,NaN,353000,000660,353000,N,NaN,353000,0,3,0.00,1546050743059,4385543,96.89,전기·전자,N,N,N,N,351500,4526286,2025-09-21 18:41:43.356719+00:00
2,3,373220,KOSPI200,NaN,NaN,Y,460500,Y,-0.85,20.00,20.00,2000,2,-1.97,347500,-5000,5,0.00,354500,-2000,5,N,,N,00,N,N,,N,N,,248500,,,NaN,NaN,NaN,354500,373220,354500,N,NaN,349500,-5000,5,-1.41,64980756750,185906,106.97,전기·전자,N,N,N,N,351500,173795,2025-09-21 18:41:43.356719+00:00
3,4,207940,KOSPI200,NaN,NaN,Y,1337000,Y,-0.58,20.00,20.00,4000,2,-0.87,1020000,-6000,5,0.10,1030000,1000,2,N,,N,00,N,N,,N,N,,721000,,,NaN,NaN,NaN,1029000,207940,1029000,N,NaN,1024000,-5000,5,-0.49,93367438000,91201,177.89,제약,N,N,N,N,1023000,51269,2025-09-21 18:41:43.356719+00:00
4,5,012450,KOSPI200,NaN,NaN,Y,1329000,Y,-1.27,20.00,20.00,17000,2,-1.76,1005000,-19000,5,1.76,1041000,12000,2,N,,N,00,N,N,,N,N,,717000,,,NaN,NaN,NaN,1023000,012450,1023000,N,NaN,1022000,-1000,5,-0.10,372660454884,363904,173.56,운송장비·부품,N,N,N,N,1010000,209665,2025-09-21 18:41:43.356719+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,093370,KOSPI200,NaN,NaN,N,8210,N,0.95,100.00,100.00,70,2,-2.53,6160,-160,5,1.11,6390,-150,5,N,,N,00,N,N,,N,N,,4430,,,NaN,NaN,NaN,6320,093370,6320,N,NaN,6230,-90,5,-1.42,3763664435,605183,50.02,화학,N,N,N,N,6380,1209836,2025-09-21 18:41:43.356719+00:00
196,197,145720,KOSPI200,NaN,NaN,Y,76100,N,-0.51,30.00,30.00,500,2,-1.02,58000,-500,5,0.68,59000,200,2,N,,N,00,N,N,,N,N,,41100,,,NaN,NaN,NaN,58600,145720,58600,N,NaN,58500,-100,5,-0.17,2245137800,38375,106.79,의료·정밀기기,N,N,N,N,58300,35934,2025-09-21 18:41:43.356719+00:00
197,198,000670,KOSPI200,NaN,NaN,N,55500,N,0.00,100.00,100.00,0,3,-1.52,42100,-1850,5,2.81,43950,-650,5,N,,N,00,N,N,,N,N,,29950,,,NaN,NaN,NaN,42750,000670,42750,N,NaN,42100,-650,5,-1.52,1123247550,26244,114.50,금속,N,N,N,N,42750,22921,2025-09-21 18:41:43.356719+00:00
198,199,456040,KOSPI200,NaN,NaN,Y,76000,N,-0.85,60.00,60.00,400,2,-1.20,57800,-300,5,0.00,58500,200,2,N,,N,00,N,N,,N,N,,41000,,,NaN,NaN,NaN,58500,456040,58500,N,NaN,58200,-300,5,-0.51,1417938050,24375,54.80,화학,N,N,N,N,58000,44483,2025-09-21 18:41:43.356719+00:00


In [3]:
final_cols = column_index[1:]

In [4]:
df[final_cols] = df[final_cols].replace(r'^\s*$', np.nan, regex=True)
df[final_cols] = df[final_cols].replace(['0', 0], np.nan)

# 결측치 개수 계산
missing_value_counts = df[final_cols].isnull().sum()

# 전체 행의 개수 계산
total_rows = len(df[final_cols])

# 결측치 비율 계산
missing_value_percent = ((missing_value_counts / total_rows) * 100).round(2)


# --- 3. 결과 취합 및 출력 ---
missing_df = pd.DataFrame({
  'missing_count': missing_value_counts,
  'missing_percent': missing_value_percent
})

# 결측치가 많은 순서대로 정렬하여 출력
missing_df = missing_df[missing_df['missing_count'] > 0]
missing_df = missing_df.sort_values(by='missing_count', ascending=False)

missing_df

,missing_count,missing_percent
divi_app_cls_code,200,100.0
ssts_hot_yn,200,100.0
short_over_cls_code,200,100.0
new_lstn_cls_name,200,100.0
vlnt_deal_cls_name,200,100.0
lwpr_vrss_prpr,21,10.5
hgpr_vrss_prpr,20,10.0
oprc_vrss_prpr,10,5.0
prdy_vrss,8,4.0


In [5]:
missing_cols = missing_df[missing_df['missing_percent'] >= 50].index.to_list()

final_cols = [col for col in final_cols if col not in missing_cols]

print(f"결측 컬럼 : {missing_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] = [후보 컬럼 : {len(final_cols)}]")

결측 컬럼 : ['divi_app_cls_code', 'ssts_hot_yn', 'short_over_cls_code', 'new_lstn_cls_name', 'vlnt_deal_cls_name']
[원본 컬럼 : 56] - [결측 컬럼 : 5] = [후보 컬럼 : 51]


In [6]:
nunique_counts = df[final_cols].nunique()

# 전체 행의 개수 계산
total_rows = len(df[final_cols])

# 결측치 비율 계산
nunique_value_percent = ((nunique_counts / total_rows) * 100).round(2)

unique_cols = nunique_counts[nunique_counts == 1].index.tolist()

# --- 3. 결과 취합 및 출력 ---
nunique_df = pd.DataFrame({
  'nunique_count': nunique_counts,
  'nunique_percent': nunique_value_percent
})

# 결측치가 많은 순서대로 정렬하여 출력
nunique_df = nunique_df[nunique_df['nunique_count'] > 1]
nunique_df = nunique_df.sort_values(by='nunique_count', ascending=False)

nunique_df.T

,ticker,bstp_cls_code,prdy_vrss_vol_rate,acml_vol,acml_tr_pbmn,prdy_vol,stck_hgpr,stck_sdpr,stck_mxpr,stck_prdy_clpr,stck_llam,stck_lwpr,stck_prpr,stck_oprc,prdy_ctrt,prdy_clpr_vrss_lwpr_rate,prdy_clpr_vrss_hgpr_rate,prdy_clpr_vrss_oprc_rate,oprc_vrss_prpr,prdy_vrss,hgpr_vrss_prpr,lwpr_vrss_prpr,bstp_kor_isnm,marg_rate,crdt_rate,oprc_vrss_prpr_sign,prdy_vrss_sign,new_hgpr_lwpr_cls_code,hgpr_vrss_prpr_sign,elw_pblc_yn,lwpr_vrss_prpr_sign,crdt_able_yn
nunique_count,200.0,200.0,200.0,200.0,200.0,200.0,197.0,195.0,195.0,195.0,191.0,190.0,190.0,187.0,166.0,160.0,142.0,127.0,110.0,108.0,76.0,75.0,21.0,6.0,6.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0
nunique_percent,100.0,100.0,100.0,100.0,100.0,100.0,98.5,97.5,97.5,97.5,95.5,95.0,95.0,93.5,83.0,80.0,71.0,63.5,55.0,54.0,38.0,37.5,10.5,3.0,3.0,1.5,1.5,1.0,1.0,1.0,1.0,1.0


In [7]:
final_cols = [col for col in final_cols if col not in unique_cols]

print(f"유니크 컬럼 : {unique_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] - [유니크 컬럼 : {len(unique_cols)}] = [후보 컬럼 : {len(final_cols)}]")

유니크 컬럼 : ['rprs_mrkt_kor_name', 'mxpr_llam_cls_code', 'mang_issu_yn', 'short_over_yn', 'mrkt_warn_cls_code', 'invt_caful_yn', 'stange_runup_yn', 'low_current_yn', 'vi_cls_code', 'flng_cls_name', 'revl_issu_reas_name', 'mrkt_warn_cls_name', 'insn_pbnt_yn', 'fcam_mod_cls_name', 'sltr_yn', 'trht_yn', 'oprc_rang_cont_yn', 'vlnt_fin_cls_code', 'updated_at']
[원본 컬럼 : 56] - [결측 컬럼 : 5] - [유니크 컬럼 : 19] = [후보 컬럼 : 32]


In [8]:
df[final_cols]

,ticker,new_hgpr_lwpr_cls_code,crdt_able_yn,stck_mxpr,elw_pblc_yn,prdy_clpr_vrss_oprc_rate,crdt_rate,marg_rate,lwpr_vrss_prpr,lwpr_vrss_prpr_sign,prdy_clpr_vrss_lwpr_rate,stck_lwpr,hgpr_vrss_prpr,hgpr_vrss_prpr_sign,prdy_clpr_vrss_hgpr_rate,stck_hgpr,oprc_vrss_prpr,oprc_vrss_prpr_sign,stck_llam,stck_sdpr,bstp_cls_code,stck_prdy_clpr,stck_prpr,prdy_vrss,prdy_vrss_sign,prdy_ctrt,acml_tr_pbmn,acml_vol,prdy_vrss_vol_rate,bstp_kor_isnm,stck_oprc,prdy_vol
0,005930,신고가,Y,104600,Y,0.75,20.00,20.00,100,2,-1.12,79600,-1500,5,0.87,81200,-1400,5,56400,80500,005930,80500,79700,-800,5,-0.99,1672547819584,20898386,81.57,전기·전자,81100,25619036
1,000660,신고가,Y,458500,Y,-0.42,20.00,20.00,5000,2,-1.42,348000,-7000,5,1.98,360000,1500,2,247500,353000,000660,353000,353000,NaN,3,0.00,1546050743059,4385543,96.89,전기·전자,351500,4526286
2,373220,NaN,Y,460500,Y,-0.85,20.00,20.00,2000,2,-1.97,347500,-5000,5,0.00,354500,-2000,5,248500,354500,373220,354500,349500,-5000,5,-1.41,64980756750,185906,106.97,전기·전자,351500,173795
3,207940,NaN,Y,1337000,Y,-0.58,20.00,20.00,4000,2,-0.87,1020000,-6000,5,0.10,1030000,1000,2,721000,1029000,207940,1029000,1024000,-5000,5,-0.49,93367438000,91201,177.89,제약,1023000,51269
4,012450,NaN,Y,1329000,Y,-1.27,20.00,20.00,17000,2,-1.76,1005000,-19000,5,1.76,1041000,12000,2,717000,1023000,012450,1023000,1022000,-1000,5,-0.10,372660454884,363904,173.56,운송장비·부품,1010000,209665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,093370,NaN,N,8210,N,0.95,100.00,100.00,70,2,-2.53,6160,-160,5,1.11,6390,-150,5,4430,6320,093370,6320,6230,-90,5,-1.42,3763664435,605183,50.02,화학,6380,1209836
196,145720,NaN,Y,76100,N,-0.51,30.00,30.00,500,2,-1.02,58000,-500,5,0.68,59000,200,2,41100,58600,145720,58600,58500,-100,5,-0.17,2245137800,38375,106.79,의료·정밀기기,58300,35934
197,000670,NaN,N,55500,N,0.00,100.00,100.00,NaN,3,-1.52,42100,-1850,5,2.81,43950,-650,5,29950,42750,000670,42750,42100,-650,5,-1.52,1123247550,26244,114.50,금속,42750,22921
198,456040,NaN,Y,76000,N,-0.85,60.00,60.00,400,2,-1.20,57800,-300,5,0.00,58500,200,2,41000,58500,456040,58500,58200,-300,5,-0.51,1417938050,24375,54.80,화학,58000,44483


In [9]:
print(final_cols)

['ticker', 'new_hgpr_lwpr_cls_code', 'crdt_able_yn', 'stck_mxpr', 'elw_pblc_yn', 'prdy_clpr_vrss_oprc_rate', 'crdt_rate', 'marg_rate', 'lwpr_vrss_prpr', 'lwpr_vrss_prpr_sign', 'prdy_clpr_vrss_lwpr_rate', 'stck_lwpr', 'hgpr_vrss_prpr', 'hgpr_vrss_prpr_sign', 'prdy_clpr_vrss_hgpr_rate', 'stck_hgpr', 'oprc_vrss_prpr', 'oprc_vrss_prpr_sign', 'stck_llam', 'stck_sdpr', 'bstp_cls_code', 'stck_prdy_clpr', 'stck_prpr', 'prdy_vrss', 'prdy_vrss_sign', 'prdy_ctrt', 'acml_tr_pbmn', 'acml_vol', 'prdy_vrss_vol_rate', 'bstp_kor_isnm', 'stck_oprc', 'prdy_vol']


| 컬럼명 | 한글컬럼명 | 중요도 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | 종목코드 | **필수** | 데이터의 Primary Key. 모든 정보를 연결하는 기준. |
| `new_hgpr_lwpr_cls_code` | 신고가/신저가 구분 | 상 | 시장의 모멘텀을 직접적으로 나타내는 중요한 기술적 지표. |
| `crdt_able_yn` | 신용거래 가능여부 | 중 | 종목의 유동성 및 안정성과 관련된 피처. |
| `stck_mxpr` | 상한가 | 중 | 가격 제한폭 정보로, 이상 급등 시나리오에 활용. |
| `elw_pblc_yn` | ELW 발행 여부 | 중 | 파생상품 존재 여부로, 변동성에 영향을 줄 수 있음. |
| `prdy_clpr_vrss_oprc_rate` | 전일종가대비시가비율 | 상 | 갭 상승/하락 여부를 판단하는 핵심 기술적 지표. |
| `crdt_rate` | 신용비율 | 중 | 종목의 신용 위험도를 나타내는 간접 지표. |
| `marg_rate` | 증거금 비율 | 중 | `crdt_rate`와 유사. 종목의 신용 위험도를 나타내는 간접 지표. |
| `lwpr_vrss_prpr` | 저가대비현재가 | 상 | 장중 저점에서의 반등 강도를 나타내는 기술적 지표. |
| `lwpr_vrss_prpr_sign` | 저가대비현재가 부호 | 상 | 저점 반등의 방향성을 나타냄. |
| `prdy_clpr_vrss_lwpr_rate` | 전일종가대비저가비율 | 상 | 전일 종가 대비 당일 저가의 위치를 나타내어 변동성 파악에 사용. |
| `stck_lwpr` | 당일 저가 | **필수** | OHLCV 데이터의 핵심 요소. 캔들차트의 기본. |
| `hgpr_vrss_prpr` | 고가대비현재가 | 상 | 장중 고점에서의 하락폭(눌림)을 나타내는 기술적 지표. |
| `hgpr_vrss_prpr_sign` | 고가대비현재가 부호 | 상 | 고점 하락의 방향성을 나타냄. |
| `prdy_clpr_vrss_hgpr_rate` | 전일종가대비고가비율 | 상 | 전일 종가 대비 당일 고가의 위치를 나타내어 변동성 파악에 사용. |
| `stck_hgpr` | 당일 고가 | **필수** | OHLCV 데이터의 핵심 요소. 캔들차트의 기본. |
| `oprc_vrss_prpr` | 시가대비현재가 | 상 | 장중 시가 대비 주가 움직임을 나타내는 핵심 지표. |
| `oprc_vrss_prpr_sign` | 시가대비현재가 부호 | 상 | 시가 대비 상승/하락 방향성을 나타냄. |
| `stck_llam` | 하한가 | 중 | 가격 제한폭 정보로, 이상 급락 시나리오에 활용. |
| `stck_sdpr` | 기준가 | 중 | 가격 변동의 기준이 되는 정보. |
| `bstp_cls_code` | 단축 종목코드 | **필수** | `ticker`와 동일한 정보로 보임. Primary Key. |
| `stck_prdy_clpr` | 전일 종가 | **필수** | 모든 '전일 대비' 계산의 기준이 되는 핵심 가격 정보. |
| `stck_prpr` | **현재가 (종가)** | **필수** | **예측 모델의 핵심 Target 또는 가장 중요한 피처.** |
| `prdy_vrss` | 전일 대비 등락 | **필수** | 주가 변동의 절대량을 나타내는 핵심 파생 피처. |
| `prdy_vrss_sign` | 전일 대비 부호 | **필수** | 주가 상승/하락 방향성을 나타내는 핵심 피처. |
| `prdy_ctrt` | 전일 대비 등락률 | **필수** | 정규화된 주가 변동률로, 모델 학습에 직접 사용. |
| `acml_tr_pbmn` | 누적 거래대금 | 상 | 시장의 관심도와 유동성을 나타내는 핵심 지표. |
| `acml_vol` | 누적 거래량 | 상 | 거래대금과 함께 시장 활성도를 판단하는 중요 지표. |
| `prdy_vrss_vol_rate` | 전일 대비 거래량 비율 | 상 | 거래량의 급증/급감을 파악하여 추세 변화를 감지. |
| `bstp_kor_isnm` | 업종명 | 상 | 산업별 트렌드 분석 및 섹터 로테이션 전략에 필수적인 피처. |
| `stck_oprc` | 당일 시가 | **필수** | OHLCV 데이터의 핵심 요소. 캔들차트의 기본. |
| `prdy_vol` | 전일 거래량 | 상 | 거래량 변화율 계산의 기준이 되는 중요 지표. |

In [10]:
final_cols = [
  'ticker',
  'new_hgpr_lwpr_cls_code',
  'crdt_able_yn',
  'stck_mxpr',
  'elw_pblc_yn',
  'prdy_clpr_vrss_oprc_rate',
  'crdt_rate',
  'marg_rate',
  'lwpr_vrss_prpr',
  'lwpr_vrss_prpr_sign',
  'prdy_clpr_vrss_lwpr_rate',
  'stck_lwpr',
  'hgpr_vrss_prpr',
  'hgpr_vrss_prpr_sign',
  'prdy_clpr_vrss_hgpr_rate',
  'stck_hgpr',
  'oprc_vrss_prpr',
  'oprc_vrss_prpr_sign',
  'stck_llam',
  'stck_sdpr',
  'bstp_cls_code',
  'stck_prdy_clpr',
  'stck_prpr',
  'prdy_vrss',
  'prdy_vrss_sign',
  'prdy_ctrt',
  'acml_tr_pbmn',
  'acml_vol',
  'prdy_vrss_vol_rate',
  'bstp_kor_isnm',
  'stck_oprc',
  'prdy_vol'
]

In [11]:
df[final_cols]

,ticker,new_hgpr_lwpr_cls_code,crdt_able_yn,stck_mxpr,elw_pblc_yn,prdy_clpr_vrss_oprc_rate,crdt_rate,marg_rate,lwpr_vrss_prpr,lwpr_vrss_prpr_sign,prdy_clpr_vrss_lwpr_rate,stck_lwpr,hgpr_vrss_prpr,hgpr_vrss_prpr_sign,prdy_clpr_vrss_hgpr_rate,stck_hgpr,oprc_vrss_prpr,oprc_vrss_prpr_sign,stck_llam,stck_sdpr,bstp_cls_code,stck_prdy_clpr,stck_prpr,prdy_vrss,prdy_vrss_sign,prdy_ctrt,acml_tr_pbmn,acml_vol,prdy_vrss_vol_rate,bstp_kor_isnm,stck_oprc,prdy_vol
0,005930,신고가,Y,104600,Y,0.75,20.00,20.00,100,2,-1.12,79600,-1500,5,0.87,81200,-1400,5,56400,80500,005930,80500,79700,-800,5,-0.99,1672547819584,20898386,81.57,전기·전자,81100,25619036
1,000660,신고가,Y,458500,Y,-0.42,20.00,20.00,5000,2,-1.42,348000,-7000,5,1.98,360000,1500,2,247500,353000,000660,353000,353000,NaN,3,0.00,1546050743059,4385543,96.89,전기·전자,351500,4526286
2,373220,NaN,Y,460500,Y,-0.85,20.00,20.00,2000,2,-1.97,347500,-5000,5,0.00,354500,-2000,5,248500,354500,373220,354500,349500,-5000,5,-1.41,64980756750,185906,106.97,전기·전자,351500,173795
3,207940,NaN,Y,1337000,Y,-0.58,20.00,20.00,4000,2,-0.87,1020000,-6000,5,0.10,1030000,1000,2,721000,1029000,207940,1029000,1024000,-5000,5,-0.49,93367438000,91201,177.89,제약,1023000,51269
4,012450,NaN,Y,1329000,Y,-1.27,20.00,20.00,17000,2,-1.76,1005000,-19000,5,1.76,1041000,12000,2,717000,1023000,012450,1023000,1022000,-1000,5,-0.10,372660454884,363904,173.56,운송장비·부품,1010000,209665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,093370,NaN,N,8210,N,0.95,100.00,100.00,70,2,-2.53,6160,-160,5,1.11,6390,-150,5,4430,6320,093370,6320,6230,-90,5,-1.42,3763664435,605183,50.02,화학,6380,1209836
196,145720,NaN,Y,76100,N,-0.51,30.00,30.00,500,2,-1.02,58000,-500,5,0.68,59000,200,2,41100,58600,145720,58600,58500,-100,5,-0.17,2245137800,38375,106.79,의료·정밀기기,58300,35934
197,000670,NaN,N,55500,N,0.00,100.00,100.00,NaN,3,-1.52,42100,-1850,5,2.81,43950,-650,5,29950,42750,000670,42750,42100,-650,5,-1.52,1123247550,26244,114.50,금속,42750,22921
198,456040,NaN,Y,76000,N,-0.85,60.00,60.00,400,2,-1.20,57800,-300,5,0.00,58500,200,2,41000,58500,456040,58500,58200,-300,5,-0.51,1417938050,24375,54.80,화학,58000,44483


In [12]:
final_df = df[final_cols].copy()

# --- 1단계: 날짜 타입 자동 변환 ---
remaining_object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in remaining_object_cols:
  original_valid_count = final_df[col].count()
  if original_valid_count == 0: continue
  
  for format in ['%Y%m%d', '%Y-%m-%d', '%Y%m', '%Y-%m']:
    date_series = pd.to_datetime(final_df[col], format=format,errors='coerce')
    success_rate = date_series.count() / original_valid_count
    
    if success_rate > 0.95: # 95% 임계값
      final_df[col] = date_series
      break

# --- 2단계: 저비율 카테고리 자동 변환 ---
object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
  ratio = final_df[col].nunique() / len(final_df[col])
  if ratio < 0.05: # 5% 임계값
    final_df[col] = final_df[col].astype('category')


In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   ticker                    200 non-null    object  
 1   new_hgpr_lwpr_cls_code    200 non-null    category
 2   crdt_able_yn              200 non-null    category
 3   stck_mxpr                 200 non-null    object  
 4   elw_pblc_yn               200 non-null    category
 5   prdy_clpr_vrss_oprc_rate  200 non-null    object  
 6   crdt_rate                 200 non-null    category
 7   marg_rate                 200 non-null    category
 8   lwpr_vrss_prpr            179 non-null    object  
 9   lwpr_vrss_prpr_sign       200 non-null    category
 10  prdy_clpr_vrss_lwpr_rate  200 non-null    object  
 11  stck_lwpr                 200 non-null    object  
 12  hgpr_vrss_prpr            180 non-null    object  
 13  hgpr_vrss_prpr_sign       200 non-null    category

| 컬럼명 | 현재 타입 | 적정 타입 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | object | category | 고유 식별자이며 종류가 한정적이므로 category가 효율적. |
| `new_hgpr_lwpr_cls_code` | category | category | 최적 타입. 범주형 데이터이므로 category 타입 유지. |
| `crdt_able_yn` | category | bool | 'Y'/'N' 값을 가지므로 bool 타입이 더 적합하고 효율적. |
| `stck_mxpr` | object | int64 | 상한가 가격은 정수형 데이터. |
| `elw_pblc_yn` | category | bool | 'Y'/'N' 값을 가지므로 bool 타입이 더 적합하고 효율적. |
| `prdy_clpr_vrss_oprc_rate` | object | float64 | 소수점을 포함하는 비율 데이터. |
| `crdt_rate` | category | float64 | 20.00 등 소수점을 포함하는 비율 데이터이므로 float64로 변환 필요. |
| `marg_rate` | category | float64 | 20.00 등 소수점을 포함하는 비율 데이터이므로 float64로 변환 필요. |
| `lwpr_vrss_prpr` | object | int64 | 가격 변동폭은 정수형 데이터. |
| `lwpr_vrss_prpr_sign` | category | category | 최적 타입. 코드성 데이터이므로 category 타입 유지. |
| `prdy_clpr_vrss_lwpr_rate` | object | float64 | 소수점을 포함하는 비율 데이터. |
| `stck_lwpr` | object | int64 | 저가 가격은 정수형 데이터. |
| `hgpr_vrss_prpr` | object | int64 | 가격 변동폭은 정수형 데이터. |
| `hgpr_vrss_prpr_sign` | category | category | 최적 타입. 코드성 데이터이므로 category 타입 유지. |
| `prdy_clpr_vrss_hgpr_rate` | object | float64 | 소수점을 포함하는 비율 데이터. |
| `stck_hgpr` | object | int64 | 고가 가격은 정수형 데이터. |
| `oprc_vrss_prpr` | object | int64 | 가격 변동폭은 정수형 데이터. |
| `oprc_vrss_prpr_sign` | category | category | 최적 타입. 코드성 데이터이므로 category 타입 유지. |
| `stck_llam` | object | int64 | 하한가 가격은 정수형 데이터. |
| `stck_sdpr` | object | int64 | 기준가 가격은 정수형 데이터. |
| `bstp_cls_code` | object | category | 종목코드와 동일한 값으로, category 타입이 효율적. |
| `stck_prdy_clpr` | object | int64 | 전일 종가 가격은 정수형 데이터. |
| `stck_prpr` | object | int64 | 현재가(종가)는 정수형 데이터. |
| `prdy_vrss` | object | int64 | 가격 변동폭은 정수형 데이터. |
| `prdy_vrss_sign` | category | category | 최적 타입. 코드성 데이터이므로 category 타입 유지. |
| `prdy_ctrt` | object | float64 | 소수점을 포함하는 등락률 데이터. |
| `acml_tr_pbmn` | object | int64 | 누적 거래대금은 매우 큰 정수. |
| `acml_vol` | object | int64 | 누적 거래량은 매우 큰 정수. |
| `prdy_vrss_vol_rate` | object | float64 | 소수점을 포함하는 비율 데이터. |
| `bstp_kor_isnm` | object | category | 업종명은 종류가 제한된 범주형 데이터. |
| `stck_oprc` | object | int64 | 시가 가격은 정수형 데이터. |
| `prdy_vol` | object | int64 | 전일 거래량은 정수형 데이터. |

In [14]:
cols_to_check = []

# 1. 모든 컬럼을 숫자 타입으로 먼저 변환
df[cols_to_check] = df[cols_to_check].apply(pd.to_numeric, errors='coerce')


# 2. 각 컬럼별로 정수 형태 데이터의 비율을 계산하는 함수를 적용
def get_integer_like_ratio(series: pd.Series) -> float:
  """
  Series의 유효한 값(NaN 제외) 중, 정수 형태인 값의 비율을 계산합니다.
  """
  # NaN 값을 제외한 데이터만 추출
  series_without_na = series.dropna()
  
  # 유효한 데이터가 없는 경우, 1.0 (100%) 반환
  if series_without_na.empty:
    return 1.0
    
  # (정수 형태인 값의 개수) / (전체 유효 데이터의 개수)
  integer_like_count = (series_without_na % 1 == 0).sum()
  total_valid_count = len(series_without_na)
  
  return integer_like_count / total_valid_count

# --- 결과 확인 ---
integer_like_ratio_result = df[cols_to_check].apply(get_integer_like_ratio)
print(integer_like_ratio_result.round(2))

Series([], dtype: float64)


In [15]:
# df.astype()에 사용하여 타입을 변경하기 위한 맵
# 이미 적정 타입인 컬럼은 제외됨
dtype_map = {
  'category': ['ticker','bstp_cls_code','bstp_kor_isnm'],
  'bool': ['crdt_able_yn','elw_pblc_yn'],
  'Int64': [
    'stck_mxpr',
    'lwpr_vrss_prpr',
    'stck_lwpr',
    'hgpr_vrss_prpr',
    'stck_hgpr',
    'oprc_vrss_prpr',
    'stck_llam',
    'stck_sdpr',
    'stck_prdy_clpr',
    'stck_prpr',
    'prdy_vrss',
    'acml_tr_pbmn',
    'acml_vol',
    'stck_oprc',
    'prdy_vol'
  ],
  'float64': [
    'prdy_clpr_vrss_oprc_rate',
    'crdt_rate',
    'marg_rate',
    'prdy_clpr_vrss_lwpr_rate',
    'prdy_clpr_vrss_hgpr_rate',
    'prdy_ctrt',
    'prdy_vrss_vol_rate'
  ]
}

for dtype, cols in dtype_map.items():
  # DataFrame에 실제 존재하는 컬럼만 필터링
  valid_cols = [col for col in cols if col in final_df.columns]
  if not valid_cols:
    continue

  try:
    if dtype == 'Int64':
      for col in valid_cols:
        numeric_col = pd.to_numeric(final_df[col], errors='coerce')
        if not numeric_col.isnull().all():
          final_df[col] = numeric_col.astype('Int64')
    
    elif dtype == 'float64':
      final_df[valid_cols] = final_df[valid_cols].apply(pd.to_numeric, errors='coerce')

    elif dtype == 'datetime64':
      for col in valid_cols:
        final_df[col] = pd.to_datetime(final_df[col], errors='coerce')
    
    else: # 'category' 등 .astype()으로 처리 가능한 나머지 타입
      final_df[valid_cols] = final_df[valid_cols].astype(dtype)

  except (ValueError, TypeError) as e:
    print(f"⚠️ 경고: 컬럼 {valid_cols}을(를) '{dtype}'으로 변환 중 오류. 건너뜁니다. (에러: {e})")


In [16]:
final_df.head()

,ticker,new_hgpr_lwpr_cls_code,crdt_able_yn,stck_mxpr,elw_pblc_yn,prdy_clpr_vrss_oprc_rate,crdt_rate,marg_rate,lwpr_vrss_prpr,lwpr_vrss_prpr_sign,prdy_clpr_vrss_lwpr_rate,stck_lwpr,hgpr_vrss_prpr,hgpr_vrss_prpr_sign,prdy_clpr_vrss_hgpr_rate,stck_hgpr,oprc_vrss_prpr,oprc_vrss_prpr_sign,stck_llam,stck_sdpr,bstp_cls_code,stck_prdy_clpr,stck_prpr,prdy_vrss,prdy_vrss_sign,prdy_ctrt,acml_tr_pbmn,acml_vol,prdy_vrss_vol_rate,bstp_kor_isnm,stck_oprc,prdy_vol
0,005930,신고가,True,104600,True,0.75,20.0,20.0,100,2,-1.12,79600,-1500,5,0.87,81200,-1400,5,56400,80500,005930,80500,79700,-800,5,-0.99,1672547819584,20898386,81.57,전기·전자,81100,25619036
1,000660,신고가,True,458500,True,-0.42,20.0,20.0,5000,2,-1.42,348000,-7000,5,1.98,360000,1500,2,247500,353000,000660,353000,353000,<NA>,3,0.00,1546050743059,4385543,96.89,전기·전자,351500,4526286
2,373220,NaN,True,460500,True,-0.85,20.0,20.0,2000,2,-1.97,347500,-5000,5,0.00,354500,-2000,5,248500,354500,373220,354500,349500,-5000,5,-1.41,64980756750,185906,106.97,전기·전자,351500,173795
3,207940,NaN,True,1337000,True,-0.58,20.0,20.0,4000,2,-0.87,1020000,-6000,5,0.10,1030000,1000,2,721000,1029000,207940,1029000,1024000,-5000,5,-0.49,93367438000,91201,177.89,제약,1023000,51269
4,012450,NaN,True,1329000,True,-1.27,20.0,20.0,17000,2,-1.76,1005000,-19000,5,1.76,1041000,12000,2,717000,1023000,012450,1023000,1022000,-1000,5,-0.10,372660454884,363904,173.56,운송장비·부품,1010000,209665


In [17]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   ticker                    200 non-null    category
 1   new_hgpr_lwpr_cls_code    200 non-null    category
 2   crdt_able_yn              200 non-null    bool    
 3   stck_mxpr                 200 non-null    Int64   
 4   elw_pblc_yn               200 non-null    bool    
 5   prdy_clpr_vrss_oprc_rate  200 non-null    float64 
 6   crdt_rate                 200 non-null    float64 
 7   marg_rate                 200 non-null    float64 
 8   lwpr_vrss_prpr            179 non-null    Int64   
 9   lwpr_vrss_prpr_sign       200 non-null    category
 10  prdy_clpr_vrss_lwpr_rate  200 non-null    float64 
 11  stck_lwpr                 200 non-null    Int64   
 12  hgpr_vrss_prpr            180 non-null    Int64   
 13  hgpr_vrss_prpr_sign       200 non-null    category